<center>
<h1> Prueba DS - MELI </h1>
    <h2> Match de productos</h2>

## Búsqueda de Productos Similares

El siguiente ejercicio tiene como objetivo encontrar productos con mayor similitud a un producto específico. Para lograr esto, necesitamos el `MLA_id` del producto que nos interesa. Utilizaremos este identificador para encontrar el artículo que tenga la mayor similitud en la categoría correspondiente.

### Pasos:

1. **Obtén el `MLA_id` del Producto de Interés:** Identifica el producto para el cual deseas encontrar productos similares. Anota el `MLA_id` que te interese de la base objetivo.

2. **Búsqueda de Similitud:** Utilizando el `MLA_id` del producto de interés, buscaremos el artículo más similar dentro de la misma categoría. Este proceso se basa en comparar características clave de los productos, como sus títulos, y encontrar aquellos que compartan patrones similares.

3. **Mejora de la Experiencia del Usuario:** Este enfoque tiene como objetivo mejorar la experiencia del usuario al ofrecer opciones similares al producto que están considerando. Al mostrar productos relacionados, aumentamos las posibilidades de que encuentren lo que están buscando de manera más eficiente.

En resumen, el objetivo de este ejercicio es ayudar a los usuarios a encontrar productos similares al que les interesa, mejorando su experiencia de compra y exploración en la plataforma.

In [85]:
import requests
import pandas as pd
import os
import json
import re
import pickle
import builtins
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize
from unidecode import unidecode
#Similitud
from sklearn.feature_extraction.text import TfidfVectorizer

In [34]:
#Validar categorias disponibles
cats = requests.get('https://api.mercadolibre.com/sites/MLA/categories')
cats.json()

[{'id': 'MLA5725', 'name': 'Accesorios para Vehículos'},
 {'id': 'MLA1512', 'name': 'Agro'},
 {'id': 'MLA1403', 'name': 'Alimentos y Bebidas'},
 {'id': 'MLA1071', 'name': 'Animales y Mascotas'},
 {'id': 'MLA1367', 'name': 'Antigüedades y Colecciones'},
 {'id': 'MLA1368', 'name': 'Arte, Librería y Mercería'},
 {'id': 'MLA1743', 'name': 'Autos, Motos y Otros'},
 {'id': 'MLA1384', 'name': 'Bebés'},
 {'id': 'MLA1246', 'name': 'Belleza y Cuidado Personal'},
 {'id': 'MLA1039', 'name': 'Cámaras y Accesorios'},
 {'id': 'MLA1051', 'name': 'Celulares y Teléfonos'},
 {'id': 'MLA1648', 'name': 'Computación'},
 {'id': 'MLA1144', 'name': 'Consolas y Videojuegos'},
 {'id': 'MLA1500', 'name': 'Construcción'},
 {'id': 'MLA1276', 'name': 'Deportes y Fitness'},
 {'id': 'MLA5726', 'name': 'Electrodomésticos y Aires Ac.'},
 {'id': 'MLA1000', 'name': 'Electrónica, Audio y Video'},
 {'id': 'MLA2547', 'name': 'Entradas para Eventos'},
 {'id': 'MLA407134', 'name': 'Herramientas'},
 {'id': 'MLA1574', 'name': 'H

In [42]:
#Funciones encargadas de generar la base sobre la cual vamos a trabajar problema de similitudes.


#Funcion que ingresa en el item unico y extrae la foto en calidad no reducida != Thumbnail

def get_image(id_sku):
    url = f'https://api.mercadolibre.com/items/{id_sku}'
    request = requests.get(url)
    item = request.json()
    
    # Limitar a solo 1 foto, opcional más a futuro, pero es necesario un array y un loop para guardarlas
    pictures = item.get('pictures', [])[:1]  
    picture_url = pictures[0].get('secure_url', None) if pictures else None
    
    #Retorna string de la url
    return picture_url



# Funcion para limpiar y estandarizar el titutlo deL producto
def clean_text(text):
    #Se eliminan tildes/acentos y carpacteres especiales
    #No se eliminan numeros con el regex
    text = unidecode(text.lower())
    text = re.sub(r'[^a-zA-Z0-9\s]', '', text)
    
    # Tokenizacion del titulo y lemmatizacion para llevar la palabras a temrinos comunes
    words = word_tokenize(text, language='spanish')
    lemmatizer = WordNetLemmatizer()
    words = [lemmatizer.lemmatize(word) for word in words]
    
    # Se eliminan stopwords en espanol
    stop_words = set(stopwords.words('spanish'))
    words = [word for word in words if word not in stop_words]
    
    cleaned_text = ' '.join(words)
    return cleaned_text

#Función que crea el dataframe con las variables necesarias
#Parámetros un Category_id y el offset
#Para temas del ejercicio se utiliza offset.
# si a futuro es necesario buscar más de 1000 registros se debe modificar parámetro en la API CON "search_type=scan"
#https://developers.mercadolibre.com.co/es_ar/items-y-busquedas
def initial_data(cat_id,offset):
    url = f'https://api.mercadolibre.com/sites/MLA/search?category={cat_id}&offset={offset}'
    request = requests.get(url)
    items = request.json()
    filtered_items = []
    #Itera sobre cada una de las ramas del json de donde nos importa sacar la infromación necesaria (Attributes)
    for item in items['results']:
        attributes = item.get('attributes', [])
        filtered_item = {
            'MLA_id': item['id'],
            'title': item['title'],
            'condition': item['condition'],
            'permalink': item['permalink'],
            'category_id': item['category_id'],
            'domain_id': item['domain_id'],
            'tags': item['tags'],
            'price': item['price'],
            'brand': next((attr['value_name'] for attr in attributes if attr['id'] == 'BRAND'), None),
            'model': next((attr['value_name'] for attr in attributes if attr['id'] == 'MODEL'), None),
            'thumbnail': item['thumbnail']
        }
        filtered_items.append(filtered_item)
    

    
    df_ini = pd.DataFrame(filtered_items)
    #aplica función de extraer la imagen a cada item del dataframe
    df_ini['url'] = df_ini.apply(lambda row: get_image(row['MLA_id']), axis=1)
    #aplica función de limpiar titulo de la publicacion
    df_ini['cleaned_title'] = df_ini.apply(lambda row: clean_text(row['title']), axis=1)
    #Resultado es un df con toda la información que se utilizará
    return df_ini


In [94]:
#Genera modelo de tfdif


#fucnión para extraer los titulos limpios de la base objetivo funcion usada en la generacion del modelo
def busca_titulos(dataframe):
    return dataframe['cleaned_title'].tolist()

def genera_modelo_tfdif(df):
    #Se asegura de estar en la misma ubicación que el notebook
    notebook_location = os.path.dirname(os.path.abspath("__file__"))
    os.chdir(notebook_location)
    
    #lee titulos de productos limpios y este es el inusmo del modelo
    names = busca_titulos(df)
    #validaremos caracteres y combinaciones hasta de 5 n-gramas
    tfidf = TfidfVectorizer(analyzer="char", ngram_range=(1, 5))
    tfidf.fit(names)
    
    model_path = os.path.join("models", "names.pkl")
    try:
        with builtins.open(model_path, "wb") as fout:
            pickle.dump(tfidf, fout)
        print("New model created and saved successfully.")
    except Exception as e:
        print(f"Error saving the model: {e}")
    
    return tfidf

In [47]:
#Base de prueba que será el objetivo

cat_id = 'MLA5725'
offset = 500
base_objetivo = initial_data(cat_id,offset)
base_objetivo.head()

In [96]:
#Objeto/producto de interés
#pendinete generar fucnión para esto para que sea mas sencillo
objeto_ini = base_objetivo.query("MLA_id=='MLA904039355'").copy()
objeto_ini

,MLA_id,title,condition,permalink,category_id,domain_id,tags,price,brand,model,thumbnail,url,cleaned_title
4,MLA904039355,Sensor Maf Caudalimetro Ecosport Fiesta 1.4 Tdci,new,https://articulo.mercadolibre.com.ar/MLA-90403...,MLA412186,MLA-VEHICLE_MAF_SENSORS,"[brand_verified, good_quality_picture, good_qu...",25611.79,UNIKAR,None,http://http2.mlstatic.com/D_846778-MLA44567101...,https://http2.mlstatic.com/D_846778-MLA4456710...,sensor maf caudalimetro ecosport fiesta 14 tdci


In [95]:

genera_modelo_tfdif(base_objetivo)

New model created and saved successfully.


TfidfVectorizer(analyzer='char', ngram_range=(1, 5))